<h1>IMDB Reviews sentiments prediction using LSTM</h1>

In [1]:
import tempfile
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as fn
import fastestimator as fe
from fastestimator.dataset.data import imdb_review
from fastestimator.op.numpyop.univariate.reshape import Reshape
from fastestimator.op.tensorop.loss import CrossEntropy
from fastestimator.op.tensorop.model import ModelOp, UpdateOp
from fastestimator.trace.io import BestModelSaver
from fastestimator.trace.metric import Accuracy

In [2]:
MAX_WORDS = 10000
MAX_LEN = 500
batch_size = 64
epochs = 10

<h2>Step 1: Prepare training & evaluation data and define Pipeline</h2>

We are loading the dataset from the tf.keras.datasets.imdb which contains movie reviews and sentiment scores. All the words have been replaced with the integers that specifies the popularity of the word in corpus. To ensure all the sequences are of same length we need to pad the input sequences before defining the Pipeline.

In [3]:
train_data, eval_data = imdb_review.load_data(MAX_LEN, MAX_WORDS)
pipeline = fe.Pipeline(train_data=train_data,
                       eval_data=eval_data,
                       batch_size=batch_size,
                       ops=Reshape(1, inputs="y", outputs="y"))

<h2>Step 2: Create model and FastEstimator network</h2>

First, we have to define the network architecture and after defining the architecture, users are expected to feed the architecture definition, its associated model name and optimizer to fe.build.

In [4]:
class ReviewSentiment(nn.Module):
    def __init__(self, embedding_size=64, hidden_units=64):
        super().__init__()
        self.embedding = nn.Embedding(MAX_WORDS, embedding_size)
        self.conv1d = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=3, padding=1)
        self.maxpool1d = nn.MaxPool1d(kernel_size=4)
        self.lstm = nn.LSTM(input_size=125, hidden_size=hidden_units, num_layers=1)
        self.fc1 = nn.Linear(in_features=hidden_units, out_features=250)
        self.fc2 = nn.Linear(in_features=250, out_features=1)

    def forward(self, x):
        x = self.embedding(x)
        x = x.permute((0, 2, 1))
        x = self.conv1d(x)
        x = fn.relu(x)
        x = self.maxpool1d(x)
        output, _ = self.lstm(x)
        x = output[:, -1]  # sequence output of only last timestamp
        x = fn.tanh(x)
        x = self.fc1(x)
        x = fn.relu(x)
        x = self.fc2(x)
        x = fn.sigmoid(x)
        return x

Network is the object that define the whole logic of neural network, including models, loss functions, optimizers etc. A Network can have several different models and loss funcitons (like GAN). <b>fe.Network</b> takes series of operators and here we feed our model in the ModelOp with inputs and outputs. It should be noted that the y_pred is the key in the data dictionary which will store the predictions.

In [5]:
model = fe.build(model_fn=lambda: ReviewSentiment(), optimizer_fn="adam")
network = fe.Network(ops=[
    ModelOp(model=model, inputs="x", outputs="y_pred"),
    CrossEntropy(inputs=("y_pred", "y"), outputs="loss"),
    UpdateOp(model=model, loss_name="loss")
])

<h2>Step 3: Prepare estimator and configure the training loop</h2>

<b>Estimator</b> is the API that wrap up the Pipeline, Network and other training metadata together. Estimator basically has four arguments network, pipeline, epochs and traces. Network and Pipeline objects are passed here as an argument. Traces are similar to the callbacks of Keras. The trace object will be called on specific timing during the training.

In the training loop, we want to measure the validation loss and save the model that has the minimum loss. BestModelSaver and Accuracy in the Trace class provide this convenient feature of storing the model.

In [6]:
model_dir = tempfile.mkdtemp()
traces = [Accuracy(true_key="y", pred_key="y_pred"), BestModelSaver(model=model, save_dir=model_dir)]
estimator = fe.Estimator(network=network,
                         pipeline=pipeline,
                         epochs=epochs,
                         traces=traces)

In [7]:
estimator.fit()

    ______           __  ______     __  _                 __            
   / ____/___ ______/ /_/ ____/____/ /_(_)___ ___  ____ _/ /_____  _____
  / /_  / __ `/ ___/ __/ __/ / ___/ __/ / __ `__ \/ __ `/ __/ __ \/ ___/
 / __/ / /_/ (__  ) /_/ /___(__  ) /_/ / / / / / / /_/ / /_/ /_/ / /    
/_/    \__,_/____/\__/_____/____/\__/_/_/ /_/ /_/\__,_/\__/\____/_/     
                                                                        

FastEstimator-Start: step: 0; model_lr: 0.001; 


/home/ubuntu/anaconda3/envs/fe_env/lib/python3.6/site-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/home/ubuntu/anaconda3/envs/fe_env/lib/python3.6/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


FastEstimator-Train: step: 0; loss: 0.69682026; 
FastEstimator-Train: step: 100; loss: 0.69128346; steps/sec: 49.53; 
FastEstimator-Train: step: 200; loss: 0.69350034; steps/sec: 51.61; 
FastEstimator-Train: step: 300; loss: 0.645782; steps/sec: 52.63; 
FastEstimator-Train: step: 391; epoch: 0; epoch_time: 7.53 sec; 
Saved model to /tmp/tmp2phwa8uv/model_best_loss.pt
FastEstimator-Eval: step: 391; epoch: 0; loss: 0.52590936; min_loss: 0.52590936; since_best: 0; accuracy: 0.7369551725784121; 
FastEstimator-Train: step: 400; loss: 0.49889562; steps/sec: 55.09; 
FastEstimator-Train: step: 500; loss: 0.4651671; steps/sec: 62.2; 
FastEstimator-Train: step: 600; loss: 0.4269103; steps/sec: 64.84; 
FastEstimator-Train: step: 700; loss: 0.46275204; steps/sec: 66.59; 
FastEstimator-Train: step: 782; epoch: 1; epoch_time: 6.15 sec; 
Saved model to /tmp/tmp2phwa8uv/model_best_loss.pt
FastEstimator-Eval: step: 782; epoch: 1; loss: 0.4363822; min_loss: 0.4363822; since_best: 0; accuracy: 0.79701150

<h2>Inferencing</h2>

We will load the best model saved during the training. First we need to create the object of model architecture (ReviewSentiment) and load the model from state_dict which contains buffers and parameters

In [8]:
model_name = 'model_best_loss.pt'
model_path = os.path.join(model_dir, model_name)
train_model = ReviewSentiment()
train_model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Get any random sequence and compare the prediction with the ground truth.

In [9]:
selected_idx = np.random.randint(10000)
print("Ground truth is: ",eval_data[selected_idx]['y'])
padded_seq = torch.from_numpy(np.array([eval_data[selected_idx]['x']]))
train_model(padded_seq)

Ground truth is:  0


tensor([[0.2048]], grad_fn=<SigmoidBackward>)